In [1]:
import torch
from fastai.text import *
from pathlib import Path
import news_utils.plot
from sklearn.metrics import f1_score, cohen_kappa_score

import pymongo
from collections import defaultdict

# print all available databases
client = pymongo.MongoClient('localhost', 27017)
cursor = client.list_databases()
for db in cursor:
    print(db)

{'name': '10000', 'sizeOnDisk': 270336.0, 'empty': False}
{'name': '10000_cl', 'sizeOnDisk': 262144.0, 'empty': False}
{'name': '10000_cl_clagreement', 'sizeOnDisk': 253952.0, 'empty': False}
{'name': '10000_cl_claudience', 'sizeOnDisk': 368640.0, 'empty': False}
{'name': '10000_cl_clcontroversial', 'sizeOnDisk': 262144.0, 'empty': False}
{'name': '10000_cl_cldisagreement', 'sizeOnDisk': 262144.0, 'empty': False}
{'name': '10000_cl_clinformative', 'sizeOnDisk': 258048.0, 'empty': False}
{'name': '10000_cl_clmean', 'sizeOnDisk': 262144.0, 'empty': False}
{'name': '10000_cl_clpersuasive', 'sizeOnDisk': 360448.0, 'empty': False}
{'name': '10000_cl_clsentiment', 'sizeOnDisk': 262144.0, 'empty': False}
{'name': '10000_cl_cltopic', 'sizeOnDisk': 262144.0, 'empty': False}
{'name': '10000_ner', 'sizeOnDisk': 274432.0, 'empty': False}
{'name': '10000_ner_cl', 'sizeOnDisk': 262144.0, 'empty': False}
{'name': '10000_ner_cl_clagreement', 'sizeOnDisk': 253952.0, 'empty': False}
{'name': '10000_ner_

In [43]:
def get_mets(db, conf=None):
#     print(db)
    mydb = client[db]
    res = mydb["metrics"].aggregate([{
        "$match": {"name": 'F1_bin'}  # only consider metric
    },
        {"$unwind": "$values"},
        {"$group":
         {'_id': '$_id',
          'val': {'$max': "$values"}, 'run_id' : { '$first': '$run_id' }}
         },  # find min values
        {"$sort": {"val": -1}}  # sort
    ])
    
    if not conf is None:
        runs = mydb['runs'].find(conf)
        runs  = [r['_id'] for r in list(runs)]
        res = [r for r in res if r['run_id'] in runs]
    
    best = list(res)[0]

    epoch = None
    max_epochs = 0

    for x in mydb['metrics'].find({'run_id': best['run_id'], 'name': 'F1_bin'}):
#         max_epochs = len(x['values'])
#         for i, v in enumerate(x['values']):
#             if v == best['val'] and epoch is None:
#                 epoch = i + 1
        f1_bin = x['values'][-1]


    for x in mydb['metrics'].find({'run_id': best['run_id'], 'name': 'prec_bin'}):
#         f1_macro = x['values'][epoch - 1]
        f1_macro = x['values'][-1]


    for x in mydb['metrics'].find({'run_id': best['run_id'], 'name': 'reca_bin'}):
        f1_micro = x['values'][-1]
#                 f1_micro = x['values'][epoch - 1]

    
    run = list(mydb['runs'].find({'_id': best['run_id']}))[0]
        
    mod = ''
    if 'mod' in  run['config']:
        mod= run['config']['mod']
        
    return f1_bin, f1_micro, f1_macro, epoch, max_epochs, run['config']['exp_id'], run['config']['drop_mult'], mod

In [44]:
all_cats = ['ArgumentsUsed', 'Discriminating', 'Inappropriate', 'OffTopic', 'PersonalStories', 'PossiblyFeedback', 'SentimentNegative', 'SentimentNeutral',  'SentimentPositive']

In [45]:
folds = range(1, 11)

In [46]:
dos = ['0_8', '0_9', '1_0']
dos = [ '0_8']

In [47]:
for c in all_cats:
    for do in dos:
        m1avg = 0
        m2avg = 0
        m3avg = 0
        for f in folds:
            try:
                m1, m2, m3, _, _, _, _, _  = get_mets(f"simple_{c}_{f}_{do}")
                m1avg += m1
                m2avg += m2
                m3avg += m3
#                 print(m1)
            except:
                pass
        m1avg /= 10
        m2avg /= 10
        m3avg /= 10
        print(c, do, m1avg, m2avg, m3avg)

ArgumentsUsed 0_8 0.6429778811439448 0.5643920475832995 0.7611650485436894
Discriminating 0_8 0.20371367169239712 0.1512386509783485 0.3288177339901478
Inappropriate 0_8 0.2720732026002481 0.20101279969784622 0.458494623655914
OffTopic 0_8 0.34318134080587104 0.277152614994593 0.4637931034482758
PersonalStories 0_8 0.7188681991866042 0.6271178349469017 0.8787775505566918
PossiblyFeedback 0_8 0.7098823158532985 0.6383226627772103 0.8178214914856137
SentimentNegative 0_8 0.6089466902840712 0.6167733989462364 0.6044448311869127
SentimentNeutral 0_8 0.6073714192708776 0.6155119038548584 0.6124908643943925
SentimentPositive 0_8 0.28312372715004297 0.2422377622377622 0.485


In [53]:
for c in all_cats:
    for do in ['1_3']:
        m1avg = 0
        m2avg = 0
        m3avg = 0
        for f in folds:
            try:
                m1, m2, m3, _, _, _, _, _  = get_mets(f"5pp_{c}_{f}_{do}")
                m1avg += m1
                m2avg += m2
                m3avg += m3
#                 print(m1)
            except:
                pass
        m1avg /= 10
        m2avg /= 10
        m3avg /= 10
        print(c, do, m1avg, m2avg, m3avg)

ArgumentsUsed 1_3 0.2873963488775598 0.22072150296971899 0.41785646297353896
Discriminating 1_3 0.03798260356180835 0.0222437636052385 0.13460591133004926
Inappropriate 1_3 0.0 0.0 0.0
OffTopic 1_3 0.0 0.0 0.0
PersonalStories 1_3 0.0 0.0 0.0
PossiblyFeedback 1_3 0.0 0.0 0.0
SentimentNegative 1_3 0.0 0.0 0.0
SentimentNeutral 1_3 0.0 0.0 0.0
SentimentPositive 1_3 0.0 0.0 0.0


In [54]:
for c in all_cats:
    for do in ['0_7']:
        m1avg = 0
        m2avg = 0
        m3avg = 0
        for f in folds:
            try:
                m1, m2, m3, _, _, _, _, _  = get_mets(f"6pp_{c}_{f}_{do}")
                m1avg += m1
                m2avg += m2
                m3avg += m3
#                 print(m1)
            except:
                pass
        m1avg /= 10
        m2avg /= 10
        m3avg /= 10
        print(c, do, m1avg, m2avg, m3avg)

ArgumentsUsed 0_7 0.6025734927381464 0.5592644053301884 0.6854940034266134
Discriminating 0_7 0.159677891671048 0.10808764990047823 0.4881773399014778
Inappropriate 0_7 0.19397317503654726 0.13582748642179154 0.4531182795698925
OffTopic 0_7 0.26039233015423363 0.21699424122583108 0.43448275862068975
PersonalStories 0_7 0.6670026514675943 0.6516007193586874 0.7329508445050368
PossiblyFeedback 0_7 0.6443466112681336 0.5362948435303245 0.8424016441573695
SentimentNegative 0_7 0.529040047907124 0.5311525470740163 0.5802227636616777
SentimentNeutral 0_7 0.508917956310627 0.5305630015174138 0.5478738954222311
SentimentPositive 0_7 0.14185539303345487 0.17976926668103138 0.32999999999999996


In [9]:
get_mets('simple_PersonalStories_4_0_9')

(0.8579881656804734,
 0.8285714285714286,
 0.8895705521472392,
 10,
 51,
 '2019_ 4_02_04_56_43_677105',
 0.9,
 'simple_fit')